## Importation librarires 

In [ ]:
import zipfile
import cv2
import numpy as np
import os 
from random import shuffle
from tqdm import tqdm
import glob
import pandas as pd
import tensorflow as tf 
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization, Input, Activation, Dropout
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import scipy
import seaborn as sns
from scipy import ndimage



In [ ]:
try:
    import scipy
    print(scipy.__version__)
    print(ndimage.affine_transform.__doc__)
    from scipy import ndimage 
except ImportError as e:
    print ('test .....', e)
    scipy = None




In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Data importattion 

In [ ]:

## Data extraction from zip file train and test data 
train_zip='../input/dogs-vs-cats-redux-kernels-edition/train.zip'
zip_ref=zipfile.ZipFile(train_zip,'r').extractall('./')

test_zip = '../input/dogs-vs-cats-redux-kernels-edition/test.zip'
zip_ref=zipfile.ZipFile(test_zip,'r').extractall('./')

In [ ]:
## Definit the data path for train and test 

TRAIN_DIR = r'./train/'

print(TRAIN_DIR)
TEST_DIR = r'./test/'

print(TRAIN_DIR)
IMG_SIZE = 100
LR = 1e-3 

## save the model name 
MODEL_NAME = 'dogvscats--{}--{}.model'.format(LR,'conv-basic')

val_loss = {}
accuracy = {}


In [ ]:
## Display the images contain in the TRAIN_DIR 
os.listdir(TRAIN_DIR)

In [ ]:
## Split class and image in TRAIN data and create a list contain image and class (target)
class_name = [i.split('.')[0] for i in os.listdir(TRAIN_DIR)]
image_name =  [i for i in os.listdir(TRAIN_DIR)]

In [ ]:
## Create a dataframe for two column class_name and image_name
df = pd.DataFrame({'class_name':class_name,'image_name':image_name} )

In [ ]:
df.head()

In this dataframe, class_name is the target take only two distinck value( cat and dog) then it classification problem 

In [ ]:
df.drop(df.index[-1], inplace=True)

In [ ]:
df

In [ ]:
## Data augmentation 
## ImageDataGenerator function from Keras_preprocessing 
train_datagen = ImageDataGenerator(
                rotation_range=15, 
                rescale=1./255,
                shear_range=0.1, 
                zoom_range=0.2,
                horizontal_flip=True, 
                width_shift_range=0.1,
                height_shift_range=0.1,
                validation_split=0.2)

In [ ]:
## Split train_fatagen into 2 train_generator and val_generator 

# since the datagen is splitted in training and validation,
# make sure to set subsets correctly

train_datagen = ImageDataGenerator(
                rescale=1./255,
                validation_split=0.2)


train_generator = train_datagen.flow_from_dataframe(
                    df,
                    TRAIN_DIR, 
                    x_col = 'image_name',
                    y_col = 'class_name',
                    target_size =(224,224),
                    class_mode = 'binary',
                    batch_size=32,
                    shuffle=False,
                    subset= 'training')
# make sure shuffle is set to false, so the predictions are done on the same order
# as they appear on the directory. batch_size should be 1 to make the
# predictions image by image 
val_generator = train_datagen.flow_from_dataframe(
                    df,
                    TRAIN_DIR, 
                    x_col = 'image_name',
                    y_col = 'class_name',
                    target_size =(224,224),
                    class_mode = 'binary',
                    batch_size=32,
                    shuffle=False,
                    subset= 'validation')

In [ ]:
# simple Convolutional layer, Dropout and GlobalMaxPooling for the sake of simplicity. You can plug your model right here and test it! Just make sure input_shape from Conv2D is the same that target_size in the generators.
## Convolution layer  
input = Input(shape=(224,224, 3))

x = Conv2D(filters=256,kernel_size=3,padding='same' )(input)
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = MaxPool2D(pool_size=2)(x)


x = Conv2D(filters=128,kernel_size=3,padding='same' )(x)
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = MaxPool2D(pool_size=2)(x)


x = Conv2D(filters=64,kernel_size=3,padding='same' )(x)
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = MaxPool2D(pool_size=2)(x)


x = Flatten()(x)
output= Dense(1, activation='sigmoid')(x)


In [ ]:
## Build the model
model = Model(input, output)

In [ ]:
model.summary()

In [ ]:
## Binary classification( cat and dog )
model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='adam')

In [ ]:
model.fit(train_generator  , validation_data=val_generator, steps_per_epoch = 128,  epochs=50)

In [ ]:
model.history.history

In [ ]:
val_loss['conv_256_128_64'] = model.history.history['val_loss']
accuracy['conv_256_128_64'] = model.history.history['accuracy']

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
input = Input(shape=(224,224, 3))

x = Conv2D(filters=256,kernel_size=3,padding='same' )(input)
x = Conv2D(filters=256,kernel_size=3,padding='same' )(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPool2D(pool_size=2)(x)


x = Conv2D(filters=128,kernel_size=3,padding='same' )(x)
x = Conv2D(filters=128,kernel_size=3,padding='same' )(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPool2D(pool_size=2)(x)


x = Conv2D(filters=64,kernel_size=3,padding='same' )(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPool2D(pool_size=2)(x)


x = Flatten()(x)
output= Dense(1, activation='sigmoid')(x)
model


model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='sgd')

model.fit(train_generator, validation_data=val_generator, steps_per_epoch = 128,  epochs=50)

In [ ]:
model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='sgd')

In [ ]:
model.fit(train_generator, validation_data=val_generator, steps_per_epoch = 128,  epochs=50)

In [ ]:
val_loss['conv_256x2_128x2_64'] = model.history.history['val_loss']
accuracy['conv_256x2_128x2_64'] = model.history.history['accuracy']

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
input = Input(shape=(224,224, 3))

x = Conv2D(filters=128,kernel_size=3, strides=2 )(input)
x = Conv2D(filters=128,kernel_size=3, strides=2 )(x)
x = BatchNormalization()(x)
x = Activation('selu')(x)
x = MaxPool2D(pool_size=2)(x)


x = Conv2D(filters=64,kernel_size=3, strides=2 )(x)
x = Conv2D(filters=64,kernel_size=3, strides=2 )(x)
x = BatchNormalization()(x)
x = Activation('selu')(x)
x = MaxPool2D(pool_size=2)(x)
x = Dropout(0.8)(x)



x = Flatten()(x)
output= Dense(1, activation='sigmoid')(x)
model = Model(input, output)

In [ ]:
model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='adam')
model.fit(train_generator, validation_data=val_generator, steps_per_epoch = 128,  epochs=50)

In [ ]:
val_loss['conv_128x2_64x2'] = model.history.history['val_loss']
accuracy['conv_128x2_64x2'] = model.history.history['accuracy']

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
input = Input(shape=(224,224, 3))


x = Conv2D(filters=64,kernel_size=3, strides=2 )(input)
x = Conv2D(filters=64,kernel_size=3, strides=2 )(x)
x = BatchNormalization()(x)
x = Activation('selu')(x)
x = MaxPool2D(pool_size=2)(x)
x = Dropout(0.8)(x)

x = Conv2D(filters=32,kernel_size=3, strides=2 )(x)
x = Conv2D(filters=32,kernel_size=3, strides=2 )(x)
x = BatchNormalization()(x)
x = Activation('selu')(x)
x = MaxPool2D(pool_size=2)(x)
x = Dropout(0.8)(x)



x = Flatten()(x)
output= Dense(1, activation='sigmoid')(x)
model = Model(input, output)


model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='adam')
model.fit(train_generator, validation_data=val_generator, steps_per_epoch = 128,  epochs=50)

In [ ]:
val_loss['conv_128x2_64x2_32x2'] = model.history.history['val_loss']
accuracy['conv_128x2_64x2_32x2'] = model.history.history['accuracy']

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:

pd.DataFrame(val_loss).plot()

## Models comparaison by accuracy 

In [ ]:

pd.DataFrame(accuracy).plot()

 
This graph compares the accuracy of different models, we see among these models, the model conv_256x2_128x2_64  appears more efficient in terms of accuracy. 



In [ ]:
input = Input(shape=(224,224, 3))

x = Conv2D(filters=256,kernel_size=3,padding='same' )(input)
x = Conv2D(filters=256,kernel_size=3,padding='same' )(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPool2D(pool_size=2)(x)


x = Conv2D(filters=128,kernel_size=3,padding='same' )(x)
x = Conv2D(filters=128,kernel_size=3,padding='same' )(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPool2D(pool_size=2)(x)


x = Conv2D(filters=64,kernel_size=3,padding='same' )(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPool2D(pool_size=2)(x)


x = Flatten()(x)
output= Dense(1, activation='sigmoid')(x)
model




In [ ]:
model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='sgd')

model.fit(train_generator, validation_data=val_generator, steps_per_epoch = 128,  epochs=50)

In [ ]:
y_pred = model.predict(val_generator)

In [ ]:
y_pred  = y_pred>0.5

In [ ]:
y_test = val_generator.classes

In [ ]:
y_pred 

In [ ]:
import seaborn as sns
sns.heatmap(tf.math.confusion_matrix(y_test, y_pred), annot=True, fmt='g').set_title("Confusion Matrix")


In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred)
auc_keras = auc(fpr_keras, tpr_keras)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='area = {:.3f}'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn import metrics
#Print classification report
print(metrics.classification_report(y_test, y_pred))

sensibility 
Specificity 
Accurancy 

In [ ]:
conf_matrix = tf.math.confusion_matrix(y_test, y_pred)

In [ ]:
# Creating a function to report confusion metrics

# save confusion matrix and slice into four pieces
TP = conf_matrix[1][1]
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
print('True Positives:', TP)
print('True Negatives:', TN)
print('False Positives:', FP)
print('False Negatives:', FN)

In [ ]:
# calculate accuracy
conf_accuracy = ((TP+TN) / (TP + TN + FP + FN))

In [ ]:
# calculate accuracy
conf_accuracy = (float(TP+TN) / float(TP + TN + FP + FN))
    
    # calculate mis-classification
conf_misclassification = 1- conf_accuracy
    
    # calculate the sensitivity
conf_sensitivity = (float(TP) / float(TP + FN))
    # calculate the specificity
conf_specificity = (float(TN) / float(TN + FP))
    
    # calculate precision
conf_precision = (float(TN) / float(TN + FP))
    # calculate f_1 score
conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
print('*'*100)
print(f'Accuracy: {round(conf_accuracy,2)}') 
print(f'Mis-Classification: {round(conf_misclassification,2)}') 
print(f'Sensitivity: {round(conf_sensitivity,2)}') 
print(f'Specificity: {round(conf_specificity,2)}') 
print(f'Precision: {round(conf_precision,2)}')
print(f'f_1 Score: {round(conf_f1,2)}')

print('*'*100)